In [0]:
import requests
import json
import cStringIO as StringIO
import gzip
from bs4 import BeautifulSoup as bs
import re

In [0]:
def search_domain(domain):
    record_list = []
    index_list = ["2019-13", "2019-09", "2019-04"]
    print ("[*] Trying target domain: %s" % domain)
    for index in index_list:
        print ("[*] Trying index %s" % index)
        cc_url  = "http://index.commoncrawl.org/CC-MAIN-%s-index?" % index
        cc_url += "url=%s&output=json" % domain
        print(unicode(cc_url, "utf-8"))
        response = requests.get(cc_url)
        if response.status_code == 200:
            records = response.content.splitlines()
            for record in records:
                record_list.append(json.loads(record))  
            print ("[*] Added %d results." % len(records))
    print ("[*] Found a total of %d hits." % len(record_list))
    return record_list

In [0]:
record_list = search_domain("www.espn.com/nba/*")

In [0]:
def download_page(record):

    offset, length = int(record['offset']), int(record['length'])
    offset_end = offset + length - 1

    # We'll get the file via HTTPS so we don't need to worry about S3 credentials
    # Getting the file on S3 is equivalent however - you can request a Range
    prefix = 'https://commoncrawl.s3.amazonaws.com/'

    # We can then use the Range header to ask for just this set of bytes
    resp = requests.get(prefix + record['filename'], headers={'Range': 'bytes={}-{}'.format(offset, offset_end)})

    # The page is stored compressed (gzip) to save space
    # We can extract it using the GZIP library
    raw_data = StringIO.StringIO(resp.content)
    f = gzip.GzipFile(fileobj=raw_data)

    # What we have now is just the WARC response, formatted:
    data = f.read()

    response = ""

    if len(data):
        try:
            warc, header, response = data.strip().split('\r\n\r\n', 2)
        except:
            pass
    #print(warc)
    #print(header)

    return response

In [0]:
i=1
for record in record_list:
  if '/story' in record['url']:
    soup = bs(download_page(record), 'html.parser')
    print("Getting Article : ", i)
    i+=1
    for p in soup.find_all("p")[0:-1]:
      file1 = open("nba.txt","a")
      file1.writelines(p.get_text().encode("utf-8"))
      file1.write("\n") 
      file1.close() 

#record_list[1]['filename'].replace('.warc', '.warc.wet').replace('/warc', '/wet')